<h1>Scrap Computer Products</h1>
First thing with only scrapy project is all about running these commands,
<ul>
<li>scrapy startproject folder_name : make a folder with scrapy struture project, then cd to the project</li>
<li>python -m venv venv : create a venv file</li>
<li>.\venv\Scripts\Activate.ps1 : to activate vene</li>
<li>scrapy genspider spider_name url_to_scrap : create spider with default parse func</li>
</ul>

And the result as default is...

In [ ]:
import scrapy


class ScrapProductsSpider(scrapy.Spider):
    name = "scrap_products"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def parse(self, response):
        links = response.xpath('')

<h3>First : Scrap all categories links</h3>

In [ ]:
def parse(self, response):
        # Scrap all links in categories
        links = response.xpath('//*[@id="res-menu"]/ul/li/a')
        print(links)

        for link in links:
            category = link.css('::text').get()
            url = link.xpath('@href').get()

            # Validate the unwanted and none response
            unwanted_titles = ['home', 'contact', 'about']

            if not category or category.lower() in unwanted_titles or not url:
                print("Skippy none title and url...")
                continue

            # Response for categories.json 
            yield {
                    'category' : category,
                    'url' : url
                }

<h3>Second : Follow Category Links and Scrap all product detail links

In [ ]:
import scrapy


class ScrapProductsSpider(scrapy.Spider):
    name = "scrap_products"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def parse(self, response):
        # Scrap all links in categories
        links = response.xpath('//*[@id="res-menu"]/ul/li/a')
        print(links)

        for link in links:
            category = link.css('::text').get()
            url = link.xpath('@href').get()

            # Validate the unwanted and none response
            unwanted_titles = ['home', 'contact', 'about']

            if not category or category.lower() in unwanted_titles or not url:
                print("Skippy none title and url...")
                continue

            # Response for categories.json 
            # yield {
            #         'category' : category,
            #         'url' : url
            #     }

            yield scrapy.Request(
                url = url,
                callback = self.parse_categories,
                meta = {'category' : category}
            )

    def parse_categories(self, response):
        category = response.meta['category']
        detail_links = response.xpath('//div[@id="content"]//div[contains(@class, "product-block product-thumb")]//a')

        products = {}
        for link in detail_links:
            product = link.css('::text').get()
            product_link = link.xpath('@href').get()

            if not product and product_link:
                print("Skipping the none product and product link...")
                continue    

            products[product] = product_link   

        yield {
                category : products
            } 
                
            


or

In [ ]:
import scrapy
import json

class ScrapProductLinksSpider(scrapy.Spider):
    name = "scrap_product_links"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def request_each_links(self):
        with open('categories_links.json', 'r') as f:
            urls = json.laod(f)
        for item in urls:
            yield scrapy.Request(url=item['url'], callback=self.parse)

    def parse(self, response):
        detail_links = response.xpath('//div[@id="content"]//div[contains(@class, "product-block product-thumb")]//a')

        for link in detail_links:
            product = link.css('::text').get()
            product_link = link.xpath('@href').get()

            if not product and product_link:
                print("Skipping the none product and product link...")
                continue
            
            yield {
                'product' : product,
                'link' : product_link
            }

